# song recommender system

In [1]:
# Step 1: Data Collection and Audio Feature Extraction

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


client_id = 'f5b49755ca164915be68aec842025edf'
client_secret = '7fc1dfcac90d4c1e9e5cfb8446426250'


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

def get_songs_from_playlist(playlist_id):
    songs = []
    results = sp.playlist_tracks(playlist_id)
    for item in results['items']:
        track = item['track']
        songs.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'id': track['id']
        })
    while results['next']:
        results = sp.next(results)
        for item in results['items']:
            track = item['track']
            songs.append({
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'id': track['id']
            })
    return songs


playlist_id = '12Wbv8sIx84T5uh6iOoJ7V'
collected_songs = get_songs_from_playlist(playlist_id)
print(f"Collected {len(collected_songs)} songs.")


def get_audio_features(songs):
    features_list = []
    for song in songs:
        features = sp.audio_features(song['id'])[0]
        if features:
            features_list.append({
                'name': song['name'],
                'artist': song['artist'],
                'id': song['id'],
                'danceability': features['danceability'],
                'energy': features['energy'],
                'key': features['key'],
                'loudness': features['loudness'],
                'mode': features['mode'],
                'speechiness': features['speechiness'],
                'acousticness': features['acousticness'],
                'instrumentalness': features['instrumentalness'],
                'liveness': features['liveness'],
                'valence': features['valence'],
                'tempo': features['tempo']
            })
    return pd.DataFrame(features_list)


collected_features_df = get_audio_features(collected_songs)
print(f"Extracted features for {len(collected_features_df)} songs.")


Collected 217 songs.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ashis\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ashis\AppData\Local\Temp\ipykernel_14260\1222921037.py", line 65, in <module>
    collected_features_df = get_audio_features(collected_songs)
  File "C:\Users\ashis\AppData\Local\Temp\ipykernel_14260\1222921037.py", line 44, in get_audio_features
    features = sp.audio_features(song['id'])[0]
  File "C:\Users\ashis\anaconda3\lib\site-packages\spotipy\client.py", line 1750, in audio_features
    results = self._get("audio-features/?ids=" + trackid)
  File "C:\Users\ashis\anaconda3\lib\site-packages\spotipy\client.py", line 327, in _get
    return self._internal_call("GET", url, payload, kwargs)
  File "C:\Users\ashis\anaconda3\lib\site-packages\spotipy\client.py", line 270, in _internal_call
    response = self._session.request(
  File "C:\Users\ashis\anaconda3\lib\site-pack

TypeError: object of type 'NoneType' has no len()

In [2]:
# Step 2: Data Loading
import pandas as pd


file_path = r"C:\Users\ashis\Downloads\data for ironheck\songs.csv"

spotify_songs_df = pd.read_csv(file_path)


print(spotify_songs_df.head())

print(f"Loaded {len(spotify_songs_df)} songs.")

   Unnamed: 0  danceability  energy  key  loudness  mode  speechiness  \
0           0         0.601   0.741    2    -5.569     1       0.0478   
1           1         0.609   0.378    4    -9.828     1       0.4380   
2           2         0.728   0.859    5    -5.237     0       0.1370   
3           3         0.601   0.843    0    -4.870     1       0.0593   
4           4         0.654   0.760    0    -3.669     0       0.0450   

   acousticness  instrumentalness  liveness  ...    tempo            type  \
0      0.185000          0.000029    0.4150  ...  153.960  audio_features   
1      0.835000          0.000000    0.2800  ...  100.418  audio_features   
2      0.040100          0.000000    0.1110  ...   65.043  audio_features   
3      0.000704          0.000053    0.0681  ...  128.002  audio_features   
4      0.079700          0.000000    0.2990  ...   99.945  audio_features   

                       id                                   uri  \
0  6I3mqTwhRpn34SLVafSH7G  spot

In [4]:
#Step 3:Combine with a Larger Dataset
import pandas as pd
import os


local_file_path = r"C:\Users\ashis\Downloads\data for ironheck\spotify songs.csv"


if os.path.exists(local_file_path):
    kaggle_songs_df = pd.read_csv(local_file_path)
    print("songs file loaded successfully.")
else:
    print("songs file not found. Please check the file path.")


collected_features_df = pd.DataFrame({
    'name': [],     
    'artist': [],
    
})


combined_songs_df = pd.concat([collected_features_df, kaggle_songs_df], ignore_index=True)


combined_songs_df.drop_duplicates(subset=['name', 'artist'], inplace=True)


print(f"Combined dataset contains {len(combined_songs_df)} songs.")


songs file loaded successfully.
Combined dataset contains 10205 songs.


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import os

# Define the file path
local_file_path = r"C:\Users\ashis\Downloads\data for ironheck\songs.csv"

# Check if the file exists
if os.path.exists(local_file_path):
    kaggle_songs_df = pd.read_csv(local_file_path)
    print("File loaded successfully.")
else:
    print("File not found. Please check the file path.")
    exit()  # Exit if the file is not found

# Assuming collected_features_df is defined somewhere
# combined_songs_df = pd.concat([collected_features_df, kaggle_songs_df], ignore_index=True)
# Ensure collected_features_df is defined; uncomment and define it accordingly

# Drop duplicates based on 'name' and 'artist'
combined_songs_df.drop_duplicates(subset=['name', 'artist'], inplace=True)
print(f"Combined dataset contains {len(combined_songs_df)} songs.")

# Check available columns
print("Available columns in combined_songs_df:")
print(combined_songs_df.columns.tolist())

# Define features for clustering
features = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Check if all features are present
if all(feature in combined_songs_df.columns for feature in features):
    # Check for missing values in features
    if combined_songs_df[features].isnull().any().any():
        print("Missing values detected in the feature columns. Please handle them before scaling.")
    else:
        # Standardize features
        scaler = StandardScaler()
        combined_songs_df[features] = scaler.fit_transform(combined_songs_df[features])

        # Apply K-means clustering
        # Change number of clusters to the number of available songs
        n_clusters = min(3, len(combined_songs_df))  # Use 3 or less based on available songs
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        combined_songs_df['cluster'] = kmeans.fit_predict(combined_songs_df[features])

        print("Clustering complete. Songs are assigned to clusters.")
else:
    print("One or more features are missing from the combined dataset.")


File loaded successfully.
Combined dataset contains 3 songs.
Available columns in combined_songs_df:
['name', 'artist', 'Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'song name']
Clustering complete. Songs are assigned to clusters.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time


client_id = 'f5b49755ca164915be68aec842025edf'
client_secret = '7fc1dfcac90d4c1e9e5cfb8446426250'
redirect_uri = 'http://localhost:9090/'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

local_file_path = r"C:\Users\ashis\Downloads\data for ironheck\songs.csv"
songs_df = pd.read_csv(local_file_path)

print("Columns in the DataFrame:", songs_df.columns)

def get_track_ids_from_df(songs_df):
    track_ids = []
    for index, row in songs_df.iterrows():
        song_name = row['song name'] 
        artist_name = row['artist']
        result = sp.search(q=f"track:{song_name} artist:{artist_name}", type='track', limit=1)

        if len(result['tracks']['items']) > 0:
            track_id = result['tracks']['items'][0]['uri']
            track_ids.append(track_id)
            print(f"Song found: {song_name} by {artist_name}")
        else:
            print(f"Track not found for {song_name} by {artist_name}.")
    return track_ids


try:
    track_ids = get_track_ids_from_df(songs_df)
except KeyError as e:
    print(f"KeyError: {e}. Please check the column names in your CSV file.")

if not track_ids:
    print("No track IDs found. Exiting...")
else:
    def fetch_audio_features(track_ids):
        audio_features_list = []
        for i in range(0, len(track_ids), 100):
            batch_ids = track_ids[i:i + 100]
            retry = True
            while retry:
                try:
                    audio_features = sp.audio_features(batch_ids)
                    audio_features_list.extend(audio_features)
                    retry = False  # Exit loop on success
                except spotipy.exceptions.SpotifyException as e:
                    if e.http_status == 429:
                        retry_after = int(e.headers.get('Retry-After', 5))
                        print(f"Rate limit hit. Retrying after {retry_after} seconds.")
                        time.sleep(retry_after)
                    else:
                        print(f"An error occurred: {e}")
                        retry = False  
        return audio_features_list


    audio_features = fetch_audio_features(track_ids)

    audio_features_df = pd.DataFrame(audio_features)

    audio_features_df.to_csv('audio_features.csv', index=False)

    print(audio_features_df.head())


Columns in the DataFrame: Index(['Unnamed: 0', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'artist', 'song name'],
      dtype='object')


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

local_file_path = r"C:\Users\ashis\Downloads\data for ironheck\songs.csv"
combined_songs_df = pd.read_csv(local_file_path)

feature_columns = ['danceability', 'energy', 'loudness']  # Ensure these columns exist in your DataFrame

sample_features = combined_songs_df[feature_columns].dropna()  # Drop NaN values if any

scaler = StandardScaler()
scaled_features = scaler.fit_transform(sample_features)

kmeans = KMeans(n_clusters=2, random_state=42)  # Adjust number of clusters as necessary
kmeans.fit(scaled_features)

combined_songs_df['cluster'] = kmeans.predict(scaled_features)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('kmeans_model.pkl', 'wb') as f:
    pickle.dump(kmeans, f)

print("Scaler and KMeans model saved successfully.")




Scaler and KMeans model saved successfully.


In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('kmeans_model.pkl', 'rb') as f:
    kmeans = pickle.load(f)

#local_file_path = r"C:\Users\ashis\Downloads\data for ironheck\songs.csv"
#combined_songs_df = pd.read_csv(local_file_path)

client_id = 'f5b49755ca164915be68aec842025edf'
client_secret = '7fc1dfcac90d4c1e9e5cfb8446426250'
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

def recommend_songs(song_name, artist_name):
    results = sp.search(q=f"{song_name} {artist_name}", limit=1, type='track')
    
    if not results['tracks']['items']:
        print("Song not found on Spotify.")
        return "Song not found on Spotify."

    track_id = results['tracks']['items'][0]['id']
    print(f"Found track ID: {track_id}")

    features = sp.audio_features([track_id])[0]
    if features is None:
        return "Features not available for the song."

    song_features = scaler.transform([[features['danceability'], features['energy'], features['loudness']]])
    cluster_label = kmeans.predict(song_features)[0]
    similar_songs = combined_songs_df[combined_songs_df['cluster'] == cluster_label]
    
    num_recommendations = min(len(similar_songs), 5)
    if num_recommendations == 0:
        return "No similar songs found in the same cluster."

    recommendations = similar_songs[['name', 'artist']].sample(num_recommendations)
    return recommendations

recommendations = recommend_songs('Shape of You', 'Ed Sheeran')
print("Recommended songs:")
print(recommendations)


Found track ID: 7qiZfU4dY1lWllzX7mPBI3


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=7qiZfU4dY1lWllzX7mPBI3:
 Max Retries, reason: too many 429 error responses

In [3]:
 sp.search(q=f"Shape of You, Ed Sheeran", limit=1, type='track')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Shape+of+You%2C+Ed+Sheeran&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'},
       'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V',
       'id': '6eUKZXaKkcviH0Ku9w2n3V',
       'name': 'Ed Sheeran',
       'type': 'artist',
       'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
   